In [11]:
##### version 6 project
# pypy/project/login_keep_credential V6-Copy1.ipynb
# Using log in form and create db file also, --------------- ok 
# Registration option included. ---------------------------- ok 
# Password Change option added ----------------------------- ok 
# show logged in User name in window ----------------------- ok
# change Password window arranged. ------------------------- ok
# all program close if logout clicked ---------------------- ok
# enter button activating for log in make easy ------------  ok       30.03.2024
# if successfully log in then get the  user interface ------ ok       30.03.2024
# print the button name which button clicked --------------- ok       30.03.2024
# error: shortcut_path not declared ======================== solved   10.04.2024
# Log in Oracle System function activating ----------------- ok       10.04.2024
# show messagebox as per click------------------------------ ok       10.04.2024
# show again the root window after log in Oracle------------ ok       10.04.2024 
# Assaign Task for create MO button------------------------- ok       10.04.2024


import tkinter as tk
from tkinter import messagebox
from tkinter.simpledialog import askstring
import pyautogui as pg
import time
import subprocess

import sqlite3
import hashlib
import tkinter as tk
from tkinter import messagebox

# Create a connection to the SQLite database
conn = sqlite3.connect('user.db')
c = conn.cursor()

# Create a table to store user information
c.execute('''CREATE TABLE IF NOT EXISTS users
             (username TEXT PRIMARY KEY, password TEXT)''')
conn.commit()

# Function to register a new user
def register():
    username = username_entry.get()
    password = password_entry.get()
    
    # Check if the username already exists
    c.execute("SELECT * FROM users WHERE username=?", (username,))
    if c.fetchone():
        messagebox.showerror("Error", "Username already exists. Please choose a different one.")
    else:
        # Hash the password before storing it
        hashed_password = hashlib.sha256(password.encode()).hexdigest()
        c.execute("INSERT INTO users (username, password) VALUES (?, ?)", (username, hashed_password))
        conn.commit()
        messagebox.showinfo("Success", "Registration successful.")

# ----------------------------------------------------- ok start-----------------


# Define global variables to store the credentials
logged_in_username = None
logged_in_password = None

# Modify the login function to store credentials after successful authentication
def login(event = None ):
    global logged_in_username, logged_in_password
    username = username_entry.get()
    password = password_entry.get()
    
    # Hash the provided password
    hashed_password = hashlib.sha256(password.encode()).hexdigest()
    # Check if the username and password match
    c.execute("SELECT * FROM users WHERE username=? AND password=?", (username, hashed_password))
    if c.fetchone():
        messagebox.showinfo("Success", "Login successful. Let's Go")
        # Store the credentials
        logged_in_username = username
        logged_in_password = password
        show_logged_in_menu(username)
        window.withdraw()  # Hide the login window after successful login
    else:
        messagebox.showerror("Error", "Invalid username or password.")

# Function to reuse stored credentials for another website or software
def reuse_credentials():
    global logged_in_username, logged_in_password
    if logged_in_username and logged_in_password:
        # Use the stored credentials to log in to another website or software
        # Replace this with your automation code to log in to the other software
        print("Logging in with stored credentials:")
        print("Username:", logged_in_username)
        print("Password:", logged_in_password)
    else:
        print("No stored credentials found. Please log in first.")





def show_logged_in_menu(username):
    global logged_in_window  # Make logged_in_window global
    def logout():
        logged_in_window.destroy()
        window.destroy()  # Close the main application window upon logout

    def on_closing():
        window.destroy()

    def show_change_password_window():
        change_password_window = tk.Toplevel(logged_in_window)
        change_password_window.title("Change Password")
        
        old_password_label = tk.Label(change_password_window, text="Old Password:")
        old_password_label.grid(row=0, column=0, padx=10, pady=5, sticky=tk.W)
        old_password_entry = tk.Entry(change_password_window, show="*")
        old_password_entry.grid(row=0, column=1, padx=10, pady=5)

        new_password_label = tk.Label(change_password_window, text="New Password:")
        new_password_label.grid(row=1, column=0, padx=10, pady=5, sticky=tk.W)
        new_password_entry = tk.Entry(change_password_window, show="*")
        new_password_entry.grid(row=1, column=1, padx=10, pady=5)

        change_button = tk.Button(change_password_window, text="Change Password", command=lambda: change_password(username, old_password_entry.get(), new_password_entry.get()))
        change_button.grid(row=2, columnspan=2, padx=10, pady=5)

    logged_in_window = tk.Toplevel(window)
    logged_in_window.title("Logged In Menu")
    
    username_label = tk.Label(logged_in_window, text="Welcome, " + username)
    username_label.pack(anchor="e", padx=10, pady=5)
    
    buttons = [
        "Log in Oracle Server",
        "Create MO",
        "Create Batch Posting",
        "Check Batch Report",
        "Download OHD",
        "Check OHD",
        "Download ILS",
        "Check ILS",
        "Check M_Transaction",
        "Change Password",  # Added Change Password button
        "Logout"
    ]
    
    for button_name in buttons:
        if button_name == "Change Password":
            button = tk.Button(logged_in_window, text=button_name, command=show_change_password_window)
        elif button_name == "Logout":
            button = tk.Button(logged_in_window, text=button_name, command=logout)
        else:
            button = tk.Button(logged_in_window, text=button_name, command=lambda btn=button_name: button_click(btn))
        button.pack(padx=10, pady=5)
    
    logged_in_window.protocol("WM_DELETE_WINDOW", on_closing)
# -*-------------------------------------------


# ----------------- function used as Sub Function --------------------------------------
firefox_shortcut_path = r"C:\ProgramData\Microsoft\Windows\Start Menu\Programs\Mozilla Firefox.lnk"

def open_firefox_dedicated_browser(firefox_shortcut_path):
    time.sleep(.2)
    logged_in_window.withdraw()  # Minimize the logged-in menu window
    time.sleep(.2)
    
    try:
        subprocess.Popen([firefox_shortcut_path], shell=True)
        print("Mozilla Firefox is now opening...")
    except Exception as e:
        print(f"Error: {e}")


def log_in_oracle_server(username, password):
    open_firefox_dedicated_browser(firefox_shortcut_path)
    # Load the webpage
    time.sleep(8)

    # Login the website
    pg.write(username)
    pg.press("tab")
    pg.write(password)
    pg.press("tab")
    pg.press("enter")

    # Wait for the page to load 
    time.sleep(2)

    logged_in_window.deiconify()  # Restore the logged-in menu window


# Function to show the change password window

def show_change_password_window(username):
    change_password_window = tk.Toplevel(window)
    change_password_window.title("Change Password")
    
    old_password_label = tk.Label(change_password_window, text="Old Password:")
    old_password_label.grid(row=0, column=0, padx=10, pady=5, sticky=tk.W)
    old_password_entry = tk.Entry(change_password_window, show="*")
    old_password_entry.grid(row=0, column=1, padx=10, pady=5)

    new_password_label = tk.Label(change_password_window, text="New Password:")
    new_password_label.grid(row=1, column=0, padx=10, pady=5, sticky=tk.W)
    new_password_entry = tk.Entry(change_password_window, show="*")
    new_password_entry.grid(row=1, column=1, padx=10, pady=5)

    change_button = tk.Button(change_password_window, text="Change Password", command=lambda: change_password(username, old_password_entry.get(), new_password_entry.get()))
    change_button.grid(row=2, columnspan=2, padx=10, pady=5)

# Function to change password
def change_password(username, old_password, new_password):
    # Hash the old password
    hashed_old_password = hashlib.sha256(old_password.encode()).hexdigest()
    
    # Check if the provided old password is correct
    c.execute("SELECT * FROM users WHERE username=? AND password=?", (username, hashed_old_password))
    if c.fetchone():
        # Hash the new password before storing it
        hashed_new_password = hashlib.sha256(new_password.encode()).hexdigest()
        c.execute("UPDATE users SET password=? WHERE username=?", (hashed_new_password, username))
        conn.commit()
        messagebox.showinfo("Success", "Password changed successfully.")
    else:
        messagebox.showerror("Error", "Invalid old password.")

# Create a tkinter window
window = tk.Tk()
window.title("Login System")

def button_click(button_name):
    global logged_in_username, logged_in_password
    if button_name == "Log in Oracle Server":
        if logged_in_username and logged_in_password:
            log_in_oracle_server(logged_in_username, logged_in_password)
        else:
            messagebox.showerror("Error", "Please log in first.")

    elif button_name == "Create MO":
        # if logged_in_username and logged_in_password:
        #     log_in_oracle_server(logged_in_username, logged_in_password)
        # else:
        #     messagebox.showerror("Error", "Please log in first.")
        
        # ----------------------------------- Create MO -----------------------------------------------------
        # Select Create Move Order Element
        def select_move_order_create():
            pg.press ("tab", presses = 12)
            pg.press ("enter")
            time.sleep (1)
            pg.press ("tab")
            pg.press ("enter")
            
        
        # Wait for the software download 
        def software_download():
            time.sleep(5)
            pg.hotkey ("alt","i")
            pg.press ("esc")
            pg.press ("enter")
            time.sleep (3)
            
        # Fill Common Data
        def fill_common_data():
            time.sleep(5)
            pg.press("tab")
            time.sleep(.2)
            pg.write ("Empty Bag for Packing Plant")
            time.sleep(.2)
            pg.press ("tab")
            
            #---------------------------------------
            time.sleep(.2)
            pg.write ("Move Order Transfer")
            pg.press ("tab")
                
            #----------------------------------------
            time.sleep(.2)
            pg.write ("CI - RM")
            pg.press ("tab")
               
            #----------------------------------------
            time.sleep(.2)
            pg.write ("CI - SFLR")
            pg.press ("tab", presses = 2)
        
        
        
        # def create_mo():
        #     time.sleep(2)
        #     # root.withdraw()  # Minimize the Tkinter window
        #     # select_move_order_create()
        #     # software_download()
        #     # fill_common_data()
        #     # messagebox.showinfo("Create MO", "Creating MO...")
        #     time.sleep(1)
        #     # root.deiconify()  # Show the Tkinter window after the login process completes
        #     messagebox.showinfo(f"{button_name}",f"Button clicked: XXCreateMOXX {button_name}")

        def create_mo():
            time.sleep(1)
            logged_in_window.withdraw()  # Minimize the logged-in menu window
            time.sleep(2)
            select_move_order_create()
            software_download()
            fill_common_data()
            time.sleep(2)
            # messagebox.showinfo("Create MO", "Creating MO...")  # Show message box after 2 seconds
            # time.sleep(.5)
            logged_in_window.deiconify()  # Restore the logged-in menu window


    
        create_mo()



    
    else:
        print(f"Button clicked: {button_name}")
        messagebox.showinfo(f"{button_name}",f"Button clicked: {button_name}")


# Username label and entry
username_label = tk.Label(window, text="Username:")
username_label.grid(row=0, column=0, padx=10, pady=5, sticky=tk.W)
username_entry = tk.Entry(window)
username_entry.grid(row=0, column=1, padx=10, pady=5)

# Password label and entry
password_label = tk.Label(window, text="Password:")
password_label.grid(row=1, column=0, padx=10, pady=5, sticky=tk.W)
password_entry = tk.Entry(window, show="*")
password_entry.grid(row=1, column=1, padx=10, pady=5)
password_entry.bind("<Return>", login)  # Bind the Enter key event to the login function

# Register and login buttons
register_button = tk.Button(window, text="Register", command=register)
register_button.grid(row=2, column=0, padx=10, pady=5)
login_button = tk.Button(window, text="Login", command=login)
login_button.grid(row=2, column=1, padx=10, pady=5)





# Start the tkinter event loop
window.mainloop()
reuse_credentials()

# Close the connection to the database
conn.close()


Mozilla Firefox is now opening...
Logging in with stored credentials:
Username: 5638
Password: Ripon
